# Statistical Analysis

## Imports

In [57]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pipeline import *


## Load Data

In [58]:
# Load the datasets
df_teams = pd.read_csv("dataset/cleaned/teams.csv")
df_teams_post = pd.read_csv("dataset/cleaned/teams_post.csv")
df_series_post = pd.read_csv("dataset/cleaned/series_post.csv")
df_players = pd.read_csv("dataset/cleaned/players.csv")
df_players_teams = pd.read_csv("dataset/cleaned/players_teams.csv")
df_coaches = pd.read_csv("dataset/cleaned/coaches.csv")
df_awards_players = pd.read_csv("dataset/cleaned/awards_players.csv")

dfs = [df_teams, df_teams_post, df_series_post, df_players, df_players_teams, df_coaches, df_awards_players]
dfs_names = ["teams", "teams_post", "series_post", "players", "players_teams", "coaches", "awards_players"]

In [59]:
df_players_merged = merge_player_info(df_players, df_players_teams) 

## Auxiliary Functions

In [60]:
def show_corr_matrix(df, threshold=0.8):
    correlation_matrix = df.corr()

    plt.figure(figsize=(12, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title("Correlation Matrix")
    plt.show()

    num_corr_greater_than_08 = ((correlation_matrix.where(np.tril(
        np.ones(correlation_matrix.shape), k=-1).astype(bool)) > threshold).sum()).sum()
    print("Number of correlation values greater than 0.8:",
          num_corr_greater_than_08)


In [61]:
def show_box_plots(df):
    num_cols = int(len(df.columns)/3)+1
    num_rows = 3
    fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(20, 10))
    axs = axs.flatten()

    for i, col in enumerate(df.columns):

        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        num_outliers = len(
            df[(df[col] < lower_bound) | (df[col] > upper_bound)])

        axs[i].boxplot(df[col])
        axs[i].set_ylim(df[col].min() - 0.1 * df[col].max(),
                        df[col].max() + 0.1 * df[col].max())

        axs[i].set_xlabel(f"mean: {df[col].mean():.2f}")

        if num_outliers > 40:
            axs[i].set_title(f"{col} ({num_outliers} outliers)", color='red')
        else:
            axs[i].set_title(f"{col} ({num_outliers} outliers)")

    fig.suptitle("Box Plots for Attributes")
    plt.tight_layout()
    plt.show()


## Statistical Analysis of the Data

In [62]:
""" 
for df, name in zip(dfs, dfs_names):

    try:
        # remove columns that have non-numeric values
        df = df.select_dtypes(include=np.number)

        show_corr_matrix(df)
        show_box_plots(df)
    except:
        print(f"Error in {name}")
        continue

# TODO: Add more statistical analysis ! 
"""


' \nfor df, name in zip(dfs, dfs_names):\n\n    try:\n        # remove columns that have non-numeric values\n        df = df.select_dtypes(include=np.number)\n\n        show_corr_matrix(df)\n        show_box_plots(df)\n    except:\n        print(f"Error in {name}")\n        continue\n\n# TODO: Add more statistical analysis ! \n'

### Statistical Players Analysis

In [63]:
# TODO: to remove year 10 add parameter 9
df_pred = player_rankings(df_players_merged)

# sort by player rankings
df_pred = df_pred.sort_values(by=['predictions'], ascending=False)

df_pred.head(10)


,playerID,medium,predictions
464,tauradi01w,3.357231,3.336905
275,leslili01w,3.303227,3.294452
86,catchta01w,3.170157,3.164172
368,parkeca01w,2.888040,2.885868
228,jacksla01w,2.698630,2.707898
379,pondeca01w,2.622344,2.642727
548,youngso01w,2.315050,2.360242
35,beardal01w,2.199414,2.206995
434,smithka01w,2.208483,2.196639
475,thompti01w,2.129342,2.120975


In [64]:
# PREDICTION IS NOT WORKING properly for now, so we use medium ranking
# df_player_evol = player_ranking_evolution(df_players_merged, 'leslili01w')

# plot of the evolution
# df_player_evol.plot(x='year', y='medium', kind='line')
# df_player_evol.head()


In [65]:
df_player_evol = player_ranking_evolution(df_players_merged, 'tauradi01w')

# plot of the evolution
df_player_evol.plot(x='year', y='medium', kind='line')
df_player_evol.head()

KeyError: "Column(s) ['award'] do not exist"

In [ ]:
# get the players in a team 

df_player_of_teams = merge_player_info(df_players, df_players_teams) 
df_players_teams = player_in_team_by_year(df_player_of_teams)
df_players_teams = team_mean(df_players_teams, df_pred)

df_players_teams.head(20)


In [ ]:
df_players_teams = df_players_teams.sort_values(by=['mean'], ascending=False)
df_players_teams.head(10)

In [ ]:


# Group the data by tmID and year and count the number of unique playerIDs for each group
df_teams_evolution = df_players_teams.groupby(['tmID', 'year']).agg({
    'mean': 'mean'
}).reset_index()


# Pivot the data to have tmID as columns and year as index
df_teams_evolution = df_teams_evolution.pivot(
    index='year', columns='tmID', values='mean')
#
# Plot the data as a stacked area chart
df_teams_evolution.plot(kind='line', stacked=False, alpha=0.5)

# Set the title and axis labels
plt.title('Evolution of Teams')
plt.xlabel('Year')
plt.ylabel('Mean Ranking')

plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

# Show the plot
plt.show()

df_teams_evolution.head(10)

# NOTA: as primeiras epocas tem menos pontuação porque tem ...
# jogadores com rankings que n são tão precisos. Uma vez que jogaram menos jogos
# TODO: podemos calcular os rankings progressivamente para um grafico mais preciso


In [ ]:
#get the teams rating for year 10 
# NOTE: I AM USING YEAR 10 ON THE PLAYER CALCULATION ... we have to take it out
year_10 = df_players_teams[df_players_teams['year'] == 10]

year_10 = year_10.sort_values(by=['mean'], ascending=False)

year_10.head(20)

In [ ]:
year_10 = df_players_teams[df_players_teams['year'] == 10]

except_year_10 = df_players_teams[df_players_teams['year'] != 10]

except_year_10 = except_year_10.groupby(['tmID']).agg({
    'mean': 'mean',
})

# EQUIPAS QUE MUDARAM DE ESTADO : UTA->SAS (epoca 4) e ORL/CON (epoca 4) TODO
# ... ver se n me esqueci de ng

except_year_10 = except_year_10.sort_values(by=['mean'], ascending=False)


except_year_10.head(50)
